In [56]:
import pandas as pd
import numpy as np
import time

In [57]:
# export DYLD_LIBRARY_PATH=$DYLD_LIBRARY_PATH:/Users/alikhannurlanuly/Progs/Progs/PySCIPOPT/lib
# conda activate
# juputer notebook

In [58]:
from pyscipopt import Model, quicksum

In [59]:
df_orders = pd.read_csv('orders.csv')
orders_length = df_orders['Length\n(au)'].values
orders_q = df_orders['Number\nof\nPieces'].values

In [60]:
df_orders

,Length\n(au),Number\nof\nPieces,Subtotal\nLength (au)
0,60,11,660
1,73,14,1022
2,117,20,2340


In [61]:
df_available = pd.read_csv('length.csv')

In [62]:
df_available

,Length\n(au),Number\nof\nPieces,Subtotal\nLength (au)
0,120,1,120
1,133,1,133
2,146,1,146
3,177,1,177
4,190,1,190
5,193,1,193
6,207,1,207
7,219,1,219
8,234,1,234
9,237,1,237


# For now take the example of fixed wire length

In [63]:
def BinPacking(w, q):
    s=[]
    for j in range(len(w)):
        for i in range(q[j]):
            s.append(w[j])
    return s

In [64]:
demand = BinPacking(orders_length, orders_q)

In [65]:
def FFD(s, B):
    remain = [B]
    sol = [[]]
    for item in sorted(s, reverse=True):
        for j,free in enumerate(remain):
            if free >= item:
                remain[j] -= item
                sol[j].append(item)
                break
        else:
            sol.append([item])
            remain.append(B-item)
    return sol

In [66]:
solution = FFD(demand, B = 250)

In [67]:
solution

[[117, 117],
 [117, 117],
 [117, 117],
 [117, 117],
 [117, 117],
 [117, 117],
 [117, 117],
 [117, 117],
 [117, 117],
 [117, 117],
 [73, 73, 73],
 [73, 73, 73],
 [73, 73, 73],
 [73, 73, 73],
 [73, 73, 60],
 [60, 60, 60, 60],
 [60, 60, 60, 60],
 [60, 60]]

In [68]:
def calculate_scrap(solution, B):
    total_scrap = 0
    for i in solution:
        total_scrap += B - sum(i)
    return total_scrap, len(solution)

In [69]:
calculate_scrap(solution, B = 250)

(478, 18)

# Optmize with Gurobi

In [70]:
def bpp(s,B):
    n = len(s)
    U = len(FFD(s,B))
    model = Model("bpp")
    x,y = {},{}
    for i in range(n):
        for j in range(U):
            x[i,j] = model.addVar(vtype="B", name="x(%s,%s)"%(i,j))
    for j in range(U):
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
    for i in range(n):
        model.addCons(quicksum(x[i,j] for j in range(U)) == 1, "Assign(%s)"%i)
    for j in range(U):
        model.addCons(quicksum(s[i]*x[i,j] for i in range(n)) <= B*y[j], "Capac(%s)"%j)
    for j in range(U):
        for i in range(n):
            model.addCons(x[i,j] <= y[j], "Strong(%s,%s)"%(i,j))
    model.setObjective(quicksum(y[j] for j in range(U)), "minimize")
    model.data = x,y
    return model

In [71]:
def solveBinPacking(s,B):
    n = len(s)
    U = len(FFD(s,B))
    model = bpp(s,B)
    x,y = model.data
    model.optimize()
    bins = [[] for i in range(U)]
    for (i,j) in x:
        if model.getVal(x[i,j]) > .5:
            bins[j].append(s[i])
    for i in range(bins.count([])):
        bins.remove([])
    for b in bins:
        b.sort()
    bins.sort()
    return bins

In [72]:
solution_bins = solveBinPacking(demand, B = 250)

In [73]:
solution_bins

[[60, 60, 117],
 [60, 73, 73],
 [60, 73, 117],
 [60, 73, 117],
 [60, 73, 117],
 [60, 73, 117],
 [60, 73, 117],
 [60, 73, 117],
 [60, 73, 117],
 [60, 73, 117],
 [73, 73, 73],
 [73, 117],
 [117, 117],
 [117, 117],
 [117, 117],
 [117, 117],
 [117, 117]]

In [74]:
calculate_scrap(solution_bins, B = 250)

(228, 17)